#  <p><center><span style="color:red"> TP 1 : Page Rank Algorithm </span></center></p>

>##       <u>     1) Practising with Python </u> 



### Construction of L and M 

Construction of a list and of it squares 

In [2]:
import numpy as np 

L=np.array([1,2,3])
M=L**2

print(L,M)

[1 2 3] [1 4 9]


### List of even integers 

Function that returns the even integers of a list 

In [3]:
def even(list):
    even=[]
    for i in list:
        if i%2==0:
            even.append(i)
    return even

even(M)

[4]

### List of integers in a file 

Function that reads the integers in a file 

In [4]:
def list_of_file(file):
    f=open(file,"r")
    list=[]
    for line in f:
        u,v= [ int(x) for x in line.split() ]
        list.append(u)
        list.append(v)
    return list 

list_of_file('graphe')

[1, 2, 2, 3, 3, 4, 4, 1, 1, 5, 5, 6, 6, 1]

## <u> 2) Implementing PageRank in Python </u>

### 1) a) Beta=1 ; eps = 0.1

First, I define two functions that will be useful to create the function PageRank

In [5]:
def distance(r1,r2): #Return the norm1 of the vector r1-r2 (to see the difference between r1 and r2)
    res=0
    n=len(r1)
    for i in range(n):
        res+=abs(r1[i]-r2[i])
    return res

def product(d,v): #Return the product of a sparse matrice represented by a dictionnary d and a vector v 
    n=len(v)
    res=np.zeros(n)
    for i in range(n):
        l=d[i+1]
        for x in l:
            res[x-1]+=(1./len(l))*v[i]
    return res

Here is the implementation of the function PageRank.<br />
It enables to apply the PageRank algorithm to a graph writtend in a file. <br />
We can adjust the Beta and the epsilon parameters. 

In [20]:
def PageRank(file,beta,eps):
    f = open(file,'r')
    d={}
    maxi=0
    for line in f: 
        print(line)  # creation of the sparse matrix, extracted from the file 
        u,v= [int(x) for x in line.split()]
        print(u,v)            # and research of the maximum of the graph's indexes maxi 
        if u in d:
            d[u].append(v)
        else:
            d[u]=[v]
        if u>maxi:
            maxi=u
        if v>maxi:
            maxi=v
    r=[]
    for i in range(maxi):
        r.append(1.0)
    r=np.array(r)                                    # creation of the initial vector result r 
    r=r/maxi
    p=(1-beta)*r
    while distance(r,beta*product(d,r)+p)>eps: 
        r=beta*product(d,r)+p                     #iteration until the distance between the old and the new r is superior to eps
    y={}
    n=len(r)
    for i in range(1,n+1):
        y["node "+str(i)]=r[i-1]
    return "There you can see the importance of each node between 0 and 1 : ", y

In [21]:
PageRank('graph',1,0.001)

1 101 301 421 111 351 261 431 41 371 211 91 11 311 412 222 422 202 432 542 372 152 92 512 32 412 142 23 173 453 303 113 353 253 123 163 93 13 33 413 144 64 524 394 424 274 404 24 224 454 114 84 234 44 94 314 504 344 484 444 244 124 434 374 174 354 14 55 525 285 125 435 335 555 515 55 35 276 66 176 396 446 386 456 246 356 86 206 266 46 336 256 376 366 156 96 326 147 527 427 547 337 147 27 277 407 77 457 237 267 47 97 417 497 347 487 447 127 297 557 377 177 467 357 368 68 398 428 338 158 278 388 458 88 238 138 258 48 98 248 438 298 378 178 368 218 329 69 309 429 339 279 229 459 389 89 239 49 99 509 499 349 489 449 129 439 379 179 359 369 219 329 19 310 1010 4810 4410 1110 3510 1210 410 3710 2110 110 310 3110 1411 1011 1111 4211 3511 5411 5511 2111 111 3112 5212 2812 4212 2612 1212 2912 3712 912 512 312 4912 3112 4013 4413 2413 4513 1313 4313 413 3613 913 514 1014 5214 714 4614 3514 4214 2614 1614 3614 3714 3214 3115 615 5215 3915 2415 815 2615 2515 1215 3315 1615 5515 3715 1515 3215 216 

ValueError: too many values to unpack (expected 2)

The node 1 is the most important in so far as two nodes target the node 1. 
The others are less important. 

### 1 )b) Beta=0.8 ; esp=0.1

We change the parameter Beta, which represents the jump possibility and we obtain : 

In [9]:
PageRank('./graph',0.8,0.001)

('There you can see the importance of each node between 0 and 1 : ',
 {'node 1': 0.27144166191245994,
  'node 2': 0.14205073558667264,
  'node 3': 0.14688009681043457,
  'node 4': 0.15069667329332564,
  'node 5': 0.14205073558667264,
  'node 6': 0.14688009681043457})

## 2) Web Graph : applying PageRank to Wikipedia pages 

Now, we have to create the graph ourself,by exploring the html code of each Wikipedia page, and by seeing if they contains links to other pages. 

In [11]:
import re
from os import listdir
from os.path import isfile, join
import html

files = [f for f in listdir("/home/paul/Téléchargements/toyset/") if isfile(join("/home/paul/Téléchargements/toyset/", f))]
n=len(files)
dico={}
for i in range(n):
    dico[i+1]=files[i]           #dico will enable to refind the name of the page with its index
    
filesSet=set(files)
graph_wiki=[]

for i in range(len(files)):
    f=open("/home/paul/Téléchargements/toyset/"+files[i],encoding="utf8")
    list_links=re.findall(r'href=[\'"]?([^\'" >]+)',f.read())    #we get the list of links in the file i 
    list_links=list(set(list_links) & filesSet)                    #we keep only those who interest us  
    for j in list_links:                                         #we append them to the grap "graph_wiki"
        v=files.index(j)
        graph_wiki.append((i+1,v+1))


FileNotFoundError: [Errno 2] No such file or directory: '/home/paul/Téléchargements/toyset/'

Now, we have our graph, named "graph_wiki". We adapt a little our function PageRank to presents the results differently. 

In [21]:
def PageRank_graph(graph,beta,eps):
    d={}
    maxi=0
    for couple in graph:                                   # creation of the sparse matrix 
        u,v= couple[0],couple[1]
        if u in d:
            d[u].append(v)
        else:
            d[u]=[v]
        if u>maxi:
            maxi=u
        if v>maxi:
            maxi=v
    r=[]
    for i in range(maxi):
        r.append(1.0)
    r=np.array(r)
    r=r/maxi
    p=(1-beta)*r
    while distance(r,beta*product(d,r)+p)>eps: 
        r=beta*product(d,r)+p #iteration until the distance between the old and the new r is superior to eps
    n=len(r)
    y={}
    for i in range(1,n+1):
        y[dico[i]]=100*r[i-1]
    return "There you can see the importance of each page between 0 and 100 : ", y

PageRank_graph(graph_wiki,1,0.1)


('There you can see the importance of each page between 0 and 100 : ',
 {'Assembly_language.html': 3.9247213058175143,
  'Binary_file.html': 0.4527633803277401,
  'Boolean_data_type.html': 0.931262282786769,
  'Bytecode.html': 1.392558988966497,
  'C++.html': 3.04362286995754,
  'COBOL.html': 2.343469619163733,
  'C_(programming_language).html': 4.065222495276738,
  'Comparison_of_programming_languages.html': 1.5043802272147742,
  'Compiler.html': 3.3443171620915737,
  'Computer.html': 2.20896294761528,
  'Computer_hardware.html': 2.2940202786461015,
  'Computer_memory.html': 1.562278964747902,
  'Computer_program.html': 2.0267176138101384,
  'Computer_science.html': 2.5251926819097723,
  'Control_flow.html': 0.9750315627785691,
  'Data_(computing).html': 0.7541860560966966,
  'Data_type.html': 1.922532815298363,
  'Database.html': 1.1228673044288302,
  'Dynamic_programming_language.html': 1.2146789950983778,
  'Executable.html': 1.2632100163216937,
  'Fortran.html': 2.417875832683193,